In [2]:

"""1.) Gather
   2.) Clean Data
        # Ensure the data is in a format that we can use to create a model
        # Missing values, numeric, standardized, leaky features - (variables are not available when we perform a prediction on a new sample, we should remove them from the model as they are leaking data from the future.)
   3.) Create Features
        # drop columns that have no variance or no signal or leak information
        # Use get_dummies for nominal categorical data with low cardinality, otherwise for high cardinality nominal data use label encoding
   4.) Sample Data, split data
        # Train and Test on different data
   5.) Impute Data with an imputer or with median values
       # Fill in data with values
   6.) Standardize/Normalize
   7.) Refactor code"""


import pandas as pd
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder
import sys
import numpy as np
import re
sys.executable


train = pd.read_csv("train.csv", index_col = "Id")
test = pd.read_csv("test.csv",index_col = "Id")

# Find columns with Missing values If the missing values in the columns are greater than a certain threshold drop them and return remaining missing columns

threshold = .5
null_cols = train.isnull().sum() 
cols_over_threshold = null_cols[null_cols > len(train) * threshold]

missing_over_threshold = list(cols_over_threshold.index)
print(f'Dropped {len(missing_over_threshold)} columns that have missing values over the specified threshold of {threshold}% \n')

# Drop columns above the specified threshold or percent_missing
train = train.drop(missing_over_threshold, axis = 1)


# Separate train features from target value
y = train.SalePrice
X = train.drop(["SalePrice"], axis = 1)

X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = .8, test_size = .2, random_state = 0)




Dropped 4 columns that have missing values over the specified threshold of 0.5% 



In [40]:

# profile = ProfileReport(train, title="Pandas Profiling Report")
# profile.to_file("train_housing_kaggle.html")


In [41]:
X_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1168.000000,956.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1162.000000,1168.000000,1168.000000,...,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,56.605308,69.614017,10589.672945,6.086473,5.572774,1970.890411,1984.692637,103.481067,439.890411,45.571918,...,473.632705,94.498288,48.044521,23.022260,3.218322,14.528253,2.118151,50.936644,6.301370,2007.819349
std,42.172322,22.946069,10704.180793,1.367472,1.116908,30.407486,20.684612,182.676225,435.106803,156.229962,...,209.442320,127.312017,68.619199,63.153093,27.916593,54.009608,36.482294,550.380636,2.725977,1.335971
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7589.500000,5.000000,5.000000,1953.750000,1966.000000,0.000000,0.000000,0.000000,...,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9512.500000,6.000000,5.000000,1972.000000,1993.000000,0.000000,379.500000,0.000000,...,477.500000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,167.750000,716.000000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,2260.000000,1120.000000,...,1390.000000,736.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [42]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 619 to 685
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1168 non-null   int64  
 1   MSZoning       1168 non-null   object 
 2   LotFrontage    956 non-null    float64
 3   LotArea        1168 non-null   int64  
 4   Street         1168 non-null   object 
 5   Alley          71 non-null     object 
 6   LotShape       1168 non-null   object 
 7   LandContour    1168 non-null   object 
 8   Utilities      1168 non-null   object 
 9   LotConfig      1168 non-null   object 
 10  LandSlope      1168 non-null   object 
 11  Neighborhood   1168 non-null   object 
 12  Condition1     1168 non-null   object 
 13  Condition2     1168 non-null   object 
 14  BldgType       1168 non-null   object 
 15  HouseStyle     1168 non-null   object 
 16  OverallQual    1168 non-null   int64  
 17  OverallCond    1168 non-null   int64  
 18  YearBui

## Find a correlation to see which features correlate the most to the house prices


In [43]:
corr = train.corr()["SalePrice"]

corr[np.argsort(corr, axis=0)[::-1]]

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64

In [58]:
## Missing Columns

"""Find columns with Missing values:
    1.) If the missing values in the columns are greater than a certain threshold drop them
    2.) For the missing values that are less than the specified threshold:
        a.) if the values are numerical, impute with mean or mode or 0 depending on the feature
        b.) if the values are categorical with low cardinalty, use one-hot encoding
        c.) If the missing values are categorical with high cardinality, use label-encoding 
"""
    
def one_hot_cols(X_train,X_valid,low_cardinality_cols,object_cols):
    """low card categorical columns using one-hot encoding"""
    print("One-Hot encoding columns") 

    # Apply one-hot encoder to each column with categorical data
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    null_cols = X_train[low_cardinality_cols].isnull().sum()
    null_cols_nums = null_cols[null_cols > 0]
    null_cols_list = list(null_cols_nums.index)
    print(X_train[null_cols_list].head(50))
    print(X_train[null_cols_list].nunique())

    pass

def label_cols(X_train,X_valid,object_cols):
    """Check to see if there are any values in X_train columns that arent in X_valid columns and handle them"""
    print("Label encoding columns")
    
#     # Check to see if there are any values in X_train columns that arent in X_valid columns

#     good_label_cols = [col for col in object_cols if 
#                        set(X_train[col]) == set(X_valid[col])]

#     # Problematic columns that will be dropped from the dataset
#     bad_label_cols = list(set(object_cols)- set(good_label_cols))

#     print('Categorical columns that will be either one-hot or label encoded:', good_label_cols)
#     print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)
    
#     # Drop categorical columns that will not be encoded
#     drop_X_train = X_train.drop(bad_label_cols, axis=1)
#     drop_X_valid = X_valid.drop(bad_label_cols, axis=1)
    
#     # Apply label encoder 
#     label_encoder = LabelEncoder()
#     for col in good_label_cols:
#         drop_X_train[col]=label_encoder.fit_transform(X_train[col])
#         drop_X_valid[col]=label_encoder.transform(X_valid[col])
#         print(drop_X_train)

    pass

def numerical_missing_cols(df,columns):
    print("Numerical or Float missing columns found")
    # convert float to integers
    
     # If the missing values are numerical, impute with mean or mode
        #df_.apply(lambda x: x.fillna(x.mean()),axis=0)
    pass
    
def check_dtypes(X_train,X_valid):
    """Check types of each column and send to respective function for further handling"""
    
    numerical_float_cols = []
    print("\nChecking dtypes of columns...\n")
    columns = [col for col in X_train.columns]

    for col in columns:
        if X_train[col].dtypes != X_valid[col].dtypes:
            raise "Column mismatch error"
        else:
            continue
                    
    print("Columns in X_train are also in X_valid") 

    
    try:
        for name,types in zip(columns,X_train[columns].dtypes):
            
            # If the dtype is object, and cardinality is higher than 10 send to low_card for further processing
            # Otherwise send to label_cols to check if the values are the same in X_train and X_valid 
            if types == "object":
                #print(name,types)
                high_cardinality = [col for col in columns if X_train[col].nunique() >= 10]
                low_cardinality = list(set(columns)-set(high_cardinality))
                #low_cardinality = [col for col in columns if X_train[col].nunique() < 10]
                     
        label_cols(X_train,X_valid,high_cardinality)
        one_hot_cols(X_train,X_valid,low_cardinality,columns)

    except Exception as Error:
        print(f"An error occured:{Error}")
        
    
check_dtypes(X_train,X_valid)

#  To calculate how many entries are added to the dataset through the one-hot encoding, begin by calculating how many entries are needed to encode the categorical variable (by multiplying the number of rows by the number of columns in the one-hot encoding). 
# Then, to obtain how many entries are added to the dataset, subtract the number of entries in the original column.




Checking dtypes of columns...

Columns in X_train are also in X_valid
Label encoding columns
One-Hot encoding columns
     GarageQual BsmtFinType2 BsmtFinType1 GarageType MasVnrType GarageFinish  \
Id                                                                             
619          TA          Unf          GLQ     Attchd    BrkFace          Unf   
871          TA          Unf          Unf     Detchd       None          Unf   
93           TA          Unf          ALQ     Detchd       None          Unf   
818          TA          Unf          GLQ     Attchd    BrkFace          RFn   
303          TA          Unf          Unf     Attchd    BrkFace          RFn   
1455         TA          Unf          GLQ     Attchd       None          RFn   
41           TA          Unf          Rec     Attchd    BrkFace          RFn   
960          TA          Unf          ALQ     Detchd       None          Unf   
76           TA          Unf          GLQ    BuiltIn       None          Unf   
1

Name: 5
Course: 4


Name      5
Course    4
dtype: int64